In [165]:
import requests
import json
import re
import pandas as pd
import numpy as np


In [158]:

def get_files(user, repo, path):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}"
    r = requests.get(url)
    data = json.loads(r.text)
    # Filter the files
    csv_files = [file for file in data if re.search(r'\d{4}-\d{2}-\d{2}-places-export\.csv', file['name'])]
    # This will return an array of all the matching files with details
    return csv_files
user = "wilfordwoodruff"
repo = "Main-Data"
path = "data/raw"
csv_files = get_files(user, repo, path)
# Sort files by date in the filename, get the latest
csv_files.sort(key=lambda x: x['name'], reverse=True)
latest_file = csv_files[0]
# Construct the raw GitHub URL
raw_url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{latest_file['path']}"
# Read the latest CSV file
df = pd.read_csv(raw_url)
# Now df holds the content of the latest CSV file
print(df.head())

   Internal ID                                    Name  Address  Country  \
0           26         Albany, Albany County, New York      NaN      NaN   
1           39  Allentown, Monmouth County, New Jersey      NaN      NaN   
2           48         Alton, Madison County, Illinois      NaN      NaN   
3          116      Apperley, Gloucestershire, England      NaN  England   
4          118                                Arkansas      NaN      NaN   

  State or Province           County       City Specific Place  \
0          New York    Albany County     Albany            NaN   
1        New Jersey  Monmouth County  Allentown            NaN   
2          Illinois   Madison County      Alton            NaN   
3               NaN  Gloucestershire   Apperley            NaN   
4               NaN              NaN        NaN            NaN   

  Modern Location Visited Mentioned Only  Latitude  Longitude  \
0             NaN     NaN            NaN       NaN        NaN   
1             Na

In [159]:
df.drop(columns = ["Latitude", "Longitude", "Country"], axis=1, inplace=True)

In [160]:
worldcities = pd.read_csv("worldcities.csv")

In [161]:
worldcities_filt = worldcities[["City_all", "Latitude", "Longitude", "Country", "admin_name"]]

In [ ]:
# # Perform the left merge using 'State or Province'
# merged_df = pd.merge(df, worldcities[['City', 'Latitude', 'Longitude']], left_on='State or Province', right_on='City', how='left', suffixes=('', '_y'))

# # Check which rows have NaN values after the first merge
# unmatched_rows = merged_df[merged_df['Latitude'].isnull()]

# # Drop the duplicate 'City_y' column from the first merge
# merged_df.drop(columns=['City_y'], inplace=True)

# # Perform the second left merge using 'County'
# merged_df = pd.merge(merged_df, worldcities[['City', 'Latitude', 'Longitude']], left_on='County', right_on='City', how='left', suffixes=('', '_county'))

# # Fill NaN values in 'Latitude' and 'Longitude' with values from the second merge
# merged_df['Latitude'].fillna(merged_df['Latitude_county'], inplace=True)
# merged_df['Longitude'].fillna(merged_df['Longitude_county'], inplace=True)

# # Drop the duplicate 'City_county' column from the second merge
# merged_df.drop(columns=['City_county', 'Latitude_county', 'Longitude_county'], inplace=True)

# # Perform the third left merge using 'City'
# final_merged_df = pd.merge(merged_df, worldcities[['City', 'Latitude', 'Longitude']], on='City', how='left', suffixes=('', '_city'))

# # Fill NaN values in 'Latitude' and 'Longitude' with values from the third merge
# final_merged_df['Latitude'].fillna(final_merged_df['Latitude_city'], inplace=True)
# final_merged_df['Longitude'].fillna(final_merged_df['Longitude_city'], inplace=True)

# # Drop the duplicate 'City_city' column from the third merge
# final_merged_df.drop(columns=['Latitude_city', 'Longitude_city'], inplace=True)

In [167]:
# Ensure 'Latitude' and 'Longitude' columns exist in 'df'
if 'Latitude' not in df.columns:
    df['Latitude'] = np.nan
if 'Longitude' not in df.columns:
    df['Longitude'] = np.nan

# Step 1: Merge on "City" and "City_all"
merged = df.merge(worldcities[['City_all', 'Latitude', 'Longitude']], left_on='City', right_on='City_all', how='left')
df['Latitude'] = merged['Latitude']
df['Longitude'] = merged['Longitude']

# Step 2: Merge on "County" for rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City_all', 'Latitude', 'Longitude']], left_on='County', right_on='City_all', how='left')
df.loc[mask, 'Latitude'] = merged['Latitude']
df.loc[mask, 'Longitude'] = merged['Longitude']

# Step 3: Merge on "Specific Place" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City_all', 'Latitude', 'Longitude']], left_on='Specific Place', right_on='City_all', how='left')
df.loc[mask, 'Latitude'] = merged['Latitude']
df.loc[mask, 'Longitude'] = merged['Longitude']

# Step 4: Merge on "Modern Location" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City_all', 'Latitude', 'Longitude']], left_on='Modern Location', right_on='City_all', how='left')
df.loc[mask, 'Latitude'] = merged['Latitude']
df.loc[mask, 'Longitude'] = merged['Longitude']

# Step 5: Merge on "State or Province" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City_all', 'Latitude', 'Longitude']], left_on='State or Province', right_on='City_all', how='left')
df.loc[mask, 'Latitude'] = merged['Latitude']
df.loc[mask, 'Longitude'] = merged['Longitude']

# Replace remaining NaN values in 'Latitude' and 'Longitude' with ":("
df['Latitude'].fillna(":(", inplace=True)
df['Longitude'].fillna(":(", inplace=True)


KeyError: 'Latitude'

In [168]:
# Step 1: Merge on "City"
merged = df.merge(worldcities[['City', 'Latitude', 'Longitude']], on='City', how='left')
df.update(merged)

# Step 2: Merge on "County" for rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City', 'Latitude', 'Longitude']], left_on='County', right_on='City', how='left')
df.update(merged)

# Step 3: Merge on "Specific Place" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City', 'Latitude', 'Longitude']], left_on='Specific Place', right_on='City', how='left')
df.update(merged)

# Step 4: Merge on "Modern Location" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City', 'Latitude', 'Longitude']], left_on='Modern Location', right_on='City', how='left')
df.update(merged)

# Step 5: Merge on "State or Province" for remaining rows where "Latitude" and "Longitude" are NaN
mask = df['Latitude'].isna() & df['Longitude'].isna()
merged = df[mask].merge(worldcities[['City', 'Latitude', 'Longitude']], left_on='State or Province', right_on='City', how='left')
df.update(merged)

# Replace remaining NaN values in 'Latitude' and 'Longitude' with ":("
df['Latitude'].fillna(":(", inplace=True)
df['Longitude'].fillna(":(", inplace=True)


KeyError: "['City'] not in index"

In [169]:
# Ensure 'Latitude' and 'Longitude' columns exist in 'df'
if 'Latitude' not in df.columns:
    df['Latitude'] = np.nan
if 'Longitude' not in df.columns:
    df['Longitude'] = np.nan

# Define columns to merge on in sequential order
merge_cols_df = ['City', 'County', 'Specific Place', 'Modern Location', 'State or Province']
merge_cols_worldcities = ['City', 'City', 'City', 'City', 'City', 'admin_name']

for df_col, wc_col in zip(merge_cols_df, merge_cols_worldcities):
    mask = df['Latitude'].isna() & df['Longitude'].isna()  # Identify rows without lat/long data
    merged = df.loc[mask].merge(worldcities[[wc_col, 'Latitude', 'Longitude']], left_on=df_col, right_on=wc_col, how='left')
    df.loc[mask, 'Latitude'] = merged['Latitude']
    df.loc[mask, 'Longitude'] = merged['Longitude']

# Replace remaining NaN values in 'Latitude' and 'Longitude' with ":("
df['Latitude'].fillna(":(", inplace=True)
df['Longitude'].fillna(":(", inplace=True)


KeyError: "['City'] not in index"

In [170]:
# Initializing ":(" to latitude and longitude columns
df['Latitude'] = ":("
df['Longitude'] = ":("

# Creating a list of columns in df to be matched
columns_to_match = ['City', 'County', 'Specific Place', 'Modern Location', 'State or Province']

# Looping through the columns to be matched
for column in columns_to_match:
    
    # Merging on each column and updating latitude and longitude where it's not NaN
    temp_df = pd.merge(df, worldcities, how='left', left_on=column, right_on='City_all')
    df.loc[temp_df['Latitude_y'].notna(), 'Latitude'] = temp_df.loc[temp_df['Latitude_y'].notna(), 'Latitude_y']
    df.loc[temp_df['Longitude_y'].notna(), 'Longitude'] = temp_df.loc[temp_df['Longitude_y'].notna(), 'Longitude_y']
    
    # Breaking the loop if all values are filled
    if (df['Latitude'] != ":(").all() and (df['Longitude'] != ":(").all():
        break

# If still there are no matches, trying with 'admin_name' in 'worldcities'
if (df['Latitude'] == ":(").any() or (df['Longitude'] == ":(").any():
    for column in columns_to_match:
        
        # Merging on each column and updating latitude and longitude where it's not NaN
        temp_df = pd.merge(df, worldcities, how='left', left_on=column, right_on='admin_name')
        df.loc[temp_df['Latitude_y'].notna(), 'Latitude'] = temp_df.loc[temp_df['Latitude_y'].notna(), 'Latitude_y']
        df.loc[temp_df['Longitude_y'].notna(), 'Longitude'] = temp_df.loc[temp_df['Longitude_y'].notna(), 'Longitude_y']
        
        # Breaking the loop if all values are filled
        if (df['Latitude'] != ":(").all() and (df['Longitude'] != ":(").all():
            break


In [171]:
df["City"].nunique()

1608

In [183]:
#2) find_missing.py

def find_missing_data(df):
    missing_values = df[df[['Latitude', 'Longitude']].isna().any(axis=1)]
    return missing_values


No missing values in locations


In [173]:
find_missing_data(df)

,Internal ID,Name,Address,State or Province,County,City,Specific Place,Modern Location,Visited,Mentioned Only,Website URL,Latitude,Longitude


No missing values in locations


In [74]:
# #3) split_dataframe.py
# import pandas as pd
# def split_dataframe(df):
#     missing_values_df = df[df[['Latitude', 'Longitude']].isna().any(axis=1)]
#     df_no_missing = df.dropna(subset=['Latitude', 'Longitude'])
#     return missing_values_df, df_no_missing

In [75]:
# #4) replace_nan.py (needs to be fixed)
# import pandas as pd
# import numpy as np
# def replace_nan_with_zero(df):
#     df['Latitude'] = df['Latitude'].apply(lambda x: 0 if pd.isnull(x) else x)
#     df['Longitude'] = df['Longitude'].apply(lambda x: 0 if pd.isnull(x) else x)
#     return df

In [ ]:
# #5) merge_dataframes.py
# import pandas as pd
# def merge_dataframes(df_no_missing, df_filled):
#     df_complete = pd.concat([df_no_missing, df_filled])
#     return df_complete

In [177]:
df

,Internal ID,Name,Address,State or Province,County,City,Specific Place,Modern Location,Visited,Mentioned Only,Website URL,Latitude,Longitude
0,26,"Albany, Albany County, New York",NaN,New York,Albany County,Albany,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/alb...,40.6943,-73.9249
1,39,"Allentown, Monmouth County, New Jersey",NaN,New Jersey,Monmouth County,Allentown,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/all...,37.8897,-122.3018
2,48,"Alton, Madison County, Illinois",NaN,Illinois,Madison County,Alton,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/alt...,31.5776,-84.1762
3,116,"Apperley, Gloucestershire, England",NaN,NaN,Gloucestershire,Apperley,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/app...,44.6272,-123.0965
4,118,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/ark...,22.3,114.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,37633,"South Kensington Museum, London, Middlesex, En...",NaN,NaN,Middlesex,London,South Kensington Museum,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/sou...,53.691,-1.633
3416,37649,"Weatogue, Simsbury, Hartford County, Connecticut",NaN,Connecticut,Hartford County,Simsbury,Weatogue,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/wea...,51.5135,-0.2707
3417,37666,"Greytown, Wairarapa, New Zealand",NaN,Wairarapa,NaN,Greytown,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/gre...,51.449,-0.337
3418,37689,"Fairview, Lincoln County, Wyoming Territory",NaN,Wyoming Territory,Lincoln County,Fairview,NaN,NaN,NaN,NaN,https://wilfordwoodruffpapers.org/subjects/fai...,54.2773,-0.4017


In [178]:
#6) main.py (not completed yet)
import pandas as pd
from import_data import get_files
from find_missing import find_missing_data
from split_dataframe import split_dataframe
from replace_nan import replace_nan_with_zero
from merge_dataframes import merge_dataframes
# Load the data
user = "wilfordwoodruff"
repo = "Main-Data"
path = "data/raw"
csv_files = get_files(user, repo, path)
# Sort files by date in the filename, get the latest
csv_files.sort(key=lambda x: x['name'], reverse=True)
latest_file = csv_files[0]
# Construct the raw GitHub URL
raw_url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{latest_file['path']}"
# Read the latest CSV file
df = pd.read_csv(raw_url)
# Extract the 'name', 'latitude', and 'longitude' columns
df = df[['Name', 'Latitude', 'Longitude']]
df['Name'] = df['Name'].str.split('|')
df = df.explode('Name')
# Find the rows where 'latitude' or 'longitude' are missing
missing_values = find_missing_data(df)
# Separate the dataframe into two; the one with the missing data and the other one with the full dataframe excluding the missing data
missing_values_df, df_no_missing = split_dataframe(df)
# Replace the NaN values with 0 values
df_filled = replace_nan_with_zero(missing_values_df)
# Merge dataframes
df_complete = merge_dataframes(df_no_missing, df_filled)
# Save the DataFrame to a new CSV file
df_complete.to_csv('new_file.csv', index=False)
# print(df_complete)

ModuleNotFoundError: No module named 'import_data'